David Omrai
20.3.2022

Data a parametry
Každý z datových souborů reprezentuje prvních cca 1000 slov nějaké anglické knihy.

!  V prvním řádku se vždy nachází informace o knize a autorovi  (tanto řádek pro analýzu ignorujte).

! V některých textech se nevyskytují všechny znaky anglické abecedy. To může být pro některé úlohy problém a je třeba se k němu postavit čelem! Může pomoct prohození souborů (tj. označení první a druhy), nebo přidání nevyskytujícího se znaku s nulovou pravděpodobností.


Zadání úlohy
1) Z obou datových souborů načtěte texty k analýze. Pro každý text zvlášť odhadněte pravděpodobnosti znaků (symbolů všetně mezery), které se v textech vyskytují. Výsledné pravděpodobnosti graficky znázorněte. 
2) Pro každý text zvlášť spočtěte entropii odhadnutého rozdělení znaků.
3) Nalezněte optimální binární instantní kód C pro kódování znaků prvního z textů.
4) Pro každý text zvlášť spočtěte střední délku kódu C a porovnejte ji s entropií rozdělení znaků. Je kód C optimální i pro druhý text?

In [2]:
# Libraries
import scipy
import numpy

In [3]:
# Load proper data file
# day of birth
K = 9
# last name length
L = len("Omrai")
X = ((K*L*23) % (20)) + 1
Y  = ((X + ((K*5 + L*7) % (19))) % (20)) + 1

file_name_x = f"{X:0=3d}.txt"
file_name_y = f"{Y:0=3d}.txt"
file_loc = "../hw1-source"

def read_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()

file_x = read_file("{}/{}".format(file_loc, file_name_x))
file_y = read_file("{}/{}".format(file_loc, file_name_y))

data_x = {}
data_y = {}

data_x["name"] = file_x.partition('\n')[0]
data_x["text"] = file_x[len(data_x["name"])+1:].strip().replace('\n', ' ')

data_y["name"] = file_y.partition('\n')[0]
data_y["text"] = file_y[len(data_y["name"])+1:].strip().replace('\n', ' ')

In [4]:
#1

# get just unique characters from texts
# uniq_chars_x = ''.join(set(data_x["text"]))
# uniq_chars_y = ''.join(set(data_y["text"]))

In [5]:
# Count the characters occurrence
def count_characters(text):
    chars_num = {}
    for i in text:
        if i not in chars_num.keys():
            chars_num[i] = {}
            chars_num[i]["num"] = 1
        else:
            chars_num[i]["num"] += 1

    text_len = len(text)
    for i in chars_num.keys():
        chars_num[i]["prob"] = chars_num[i]["num"] / text_len

    return chars_num

In [6]:
uniq_num_x = count_characters(data_x["text"])
uniq_num_y = count_characters(data_y["text"])

In [7]:
print(uniq_num_x.keys())
print(uniq_num_y.keys())

dict_keys(['t', 'h', 'e', ' ', 'r', 'p', 'u', 'b', 'l', 'i', 'c', 's', 'o', 'n', 'f', 'd', 'a', 'g', 'w', 'm', 'y', 'k', 'v', 'q', 'j', 'x'])
dict_keys(['t', 'h', 'e', ' ', 'c', 'i', 'l', 'd', 'r', 'n', 'a', 'm', 'p', 'o', 'f', 's', 'u', 'g', 'b', 'y', 'w', 'v', 'k', 'q', 'j', 'x', 'z'])


In [9]:
# Add the rest of english characters
eng_chars = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

def add_missing_chars(chars, uniq_dict):
    for char in chars:
        if (char not in uniq_dict.keys()):
            uniq_dict[char] = {}
            uniq_dict[char]["num"] = 0
            uniq_dict[char]["prob"] = 0.0

add_missing_chars(eng_chars, uniq_num_x)
add_missing_chars(eng_chars, uniq_num_y)

add_missing_chars(uniq_num_x.keys(), uniq_num_y)
add_missing_chars(uniq_num_y.keys(), uniq_num_x)



{'t': {'num': 453, 'prob': 0.07127123977344242}, 'h': {'num': 344, 'prob': 0.054122089364380115}, 'e': {'num': 638, 'prob': 0.10037759597230964}, ' ': {'num': 1249, 'prob': 0.19650723725613595}, 'r': {'num': 284, 'prob': 0.0446821900566394}, 'p': {'num': 91, 'prob': 0.014317180616740088}, 'u': {'num': 189, 'prob': 0.02973568281938326}, 'b': {'num': 58, 'prob': 0.009125235997482694}, 'l': {'num': 192, 'prob': 0.030207677784770296}, 'i': {'num': 321, 'prob': 0.05050346129641284}, 'c': {'num': 151, 'prob': 0.023757079924480805}, 's': {'num': 331, 'prob': 0.05207677784770296}, 'o': {'num': 427, 'prob': 0.0671806167400881}, 'n': {'num': 310, 'prob': 0.048772813089993705}, 'f': {'num': 100, 'prob': 0.015733165512901194}, 'd': {'num': 205, 'prob': 0.03225298930144745}, 'a': {'num': 455, 'prob': 0.07158590308370044}, 'g': {'num': 91, 'prob': 0.014317180616740088}, 'w': {'num': 132, 'prob': 0.02076777847702958}, 'm': {'num': 135, 'prob': 0.021239773442416614}, 'y': {'num': 131, 'prob': 0.020610